Force colab into high ram state.

In [0]:
a = []
while(1):
    a.append('1')

KeyboardInterrupt: ignored

In [0]:
###installation of mne for colab only don't worry about Errors for now
#!wget "https://raw.githubusercontent.com/mne-tools/mne-python/master/requirements.txt"
#!pip install -r "/content/requirements.txt"
!pip install -U mne

#!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 6.4MB 2.5MB/s 
     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 450kB 48.6MB/s 
     |████████████████████████████████| 3.8MB 28.9MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
%matplotlib inline
import mne
import numpy as np
import pandas as pd
import os
import sys
import csv
import zipfile
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt


In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "/content/gdrive/Shared drives/CSE140/EDFFiles/DATA.zip"


In [0]:
zip_ref = zipfile.ZipFile(path, 'r')
#zip_ref.extractall("/content/drive/Shared drives/CSE140/Data/")
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
tmpdir = '/tmp/DATA/'
s_files = ['S01/', 'S02/', 'S03/', 'S04/', 'S05/', 'S06/', 'S07/', 'S08/', 'S09/', 'S10/', 'S11/', 'S12/', 'S13/', 'S14/', 'S15/', 'S16/', 'S17/']
sessions= ['Session1/', 'Session2/']

In [0]:
sensorsPosition = []
with open("/tmp/DATA/sensorsPosition.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader: # each row is a list
        sensorsPosition.append([float(row[2]), float(row[3]), float(row[4])])

In [0]:
#Get the provided bad channels.
def getBadChannelArray(path):
    results = []
    with open(path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            results.append(row[0])
            
    return results
            #results.append([float(row[2]), float(row[3]), float(row[4])])

In [0]:
#Helper function for normalizing data.
def normalizeData(edfpath, bads, ann):
  array = []

  raw = mne.io.read_raw_edf(edfpath,preload=True)

  #Set bad channels to the provided bad channels.
  raw.info['bads'] = bads

  raw.drop_channels('Diff 2')
  raw.drop_channels('Diff 3')
  raw.drop_channels('Diff 4')

  for i, j in zip(sensorsPosition, raw.info['chs']):
    j['loc'] = i
  
  #Interpolate bad channels so that they are an average of neighbors.
  raw.interpolate_bads()

  #filtered_raw = raw.filter(None, None, None)

  with open(ann) as csvfile:
    reader = csv.reader(csvfile)
    next(reader)

    #Each row is a csv value in a csv file of: label, video_source, onset_start, onset_end
    for row in reader: #
      if row == []:
        continue

      onset_start = float(row[2]) + 0.5
      onset_end = onset_start + 5.0

      #Cut EEG data into 5 second segments.
      time_slice = raw.copy().crop( tmin=onset_start, tmax=onset_end,include_tmax=False)
      print(time_slice)

      #Convert to numpy array.
      arr = time_slice.get_data()

      #Compute FFT of numpy array.
      x = tf.convert_to_tensor( arr, dtype=tf.float32 )
      xt = tf.signal.rfft(x, fft_length=[500*5], name=None)
      xt = tf.abs(xt) 
      xn = np.array(xt)
      #Pull first 128 values.
      xn = xn[:,1:129]
      print(np.shape(xn))
      array.append([xn, int(row[0])])
  
  return array 






  



In [0]:
#Actual execution of normalizing data:
#  Loop through entire dataset for each EEG file:
#    Interpolate the provided bad channels.
#    Chop file based on given time intervals.
#    Compute FFTs on time intervals.
#    Append each (FFT, label) to dataset.
#    Repeat.
dataset = []
for s_file_number in s_files:
  bad_channel_path = str(tmpdir) + str(s_file_number) + "BAD_CHANNELS.txt"
  bad_channels = getBadChannelArray(bad_channel_path)

  for session_number in sessions:
    path = str(tmpdir) + str(s_file_number) + str(session_number)
    for file in os.listdir(path):
      if file.endswith(".edf"):
        edfpath = os.path.join(path, file)
        csvpath = os.path.join(path, "ann.csv")
        normal_data = normalizeData(edfpath, bad_channels, csvpath)
        dataset.append(normal_data)
        print('Done with: ', file)

#save_path = '/content/gdrive/My Drive/CSE140/TrainRevamped.npy'
#np.save(save_path, dataset)

      

Extracting EDF parameters from /tmp/DATA/S01/Session1/NeoRec_2018-10-04_13-28-04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 645499  =      0.000 ...  1290.998 secs...
Computing interpolation matrix from 126 sensor positions
Interpolating 2 sensors
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-04_13-28-04.edf, n_channels x n_times : 128 x 2500 (5.

<ipython-input-9-550dfff7d691>:14: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_16-50-24.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-

<ipython-input-9-550dfff7d691>:14: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-15_17-25-48.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-

...

<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-02_11-16-51.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-

<ipython-input-9-550dfff7d691>:14: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-01-09.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-

<ipython-input-9-550dfff7d691>:14: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-10-23_13-27-01.edf, n_channels x n_times : 128 x 2500 (5.0 sec), ~2.7 MB, data loaded>
(128, 128)
<RawEDF  |  NeoRec_2018-

In [0]:
temp = dataset
normal_set = np.reshape(temp, (3978, 2))
np.random.shuffle(normal_set)

In [0]:
X = np.array([i[0] for i in normal_set])
Y = np.array([i[1] for i in normal_set])

In [0]:
save_path = '/content/gdrive/My Drive/CSE140/128Data.npy'
np.save(save_path, X)

In [0]:
save_path = '/content/gdrive/My Drive/CSE140/128Label.npy'
np.save(save_path, Y)